In [ ]:
import pybroker as pb
from pybroker import highv
from pybroker import Strategy, StrategyConfig, ExecContext
from pybroker.ext.data import AKShare
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from numba import njit
from pybroker import IndicatorSet
import talib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import akshare as ak

def train_macd(symbol, train_data, test_data):
    # Train
    # Previous day close prices.
    # 1   4   5   7
    # nan 1   4   5
    train_prev_close = train_data['close'].shift(1)
    # Calculate daily returns.
    train_daily_returns = (train_data['close'] - train_prev_close) / train_prev_close
    # Predict next day's return.
    train_data['pred'] = train_daily_returns.shift(-1)
    train_data = train_data.dropna()
    # Train the LinearRegession model to predict the next day's return
    # given the 20-day CMMA.

    # X_train = (train_data['macd_dif']-train_data['macd_dea']).to_frame("macd_dif")
    X_train =  train_data[['macd_dif','macd_dea',"macd_hist"]]
    y_train = train_data[['pred']]
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Test
    test_prev_close = test_data['close'].shift(1)
    test_daily_returns = (test_data['close'] - test_prev_close) / test_prev_close
    test_data['pred'] = test_daily_returns.shift(-1)
    test_data = test_data.dropna()

    # X_test = (test_data['macd_dif']-test_data['macd_dea']).to_frame("macd_dif")
    X_test = test_data[['macd_dif','macd_dea','macd_hist']]
    y_test = test_data[['pred']]
    # Make predictions from test data.
    y_pred = model.predict(X_test)
    # Print goodness of fit.
    r2 = r2_score(y_test, np.squeeze(y_pred))
    print(symbol, f'R^2={r2}')

    # Return the trained model and columns to use as input data.
    return model, ['macd_dif',"macd_dea","macd_hist"]

def cmma(bar_data, lookback):

    @njit  # Enable Numba JIT.
    def vec_cmma(values):
        # Initialize the result array.
        n = len(values)
        out = np.array([np.nan for _ in range(n)])

        # For all bars starting at lookback:
        for i in range(lookback, n):
            # Calculate the moving average for the lookback.
            ma = 0
            for j in range(i - lookback, i):
                ma += values[j]
            ma /= lookback
            # Subtract the moving average from value.
            out[i] = values[i] - ma
        return out
    # Calculate with close prices.
    return vec_cmma(bar_data.close)
config = StrategyConfig(bootstrap_sample_size=100)

strategy = Strategy(
    data_source=AKShare(),
    start_date='20240101',
    end_date=datetime.now(),
    config=config
)
def hold_long(ctx):
    if not ctx.long_pos():
        # Buy if the next bar is predicted to have a positive return:
        if ctx.preds('macdModel')[-1] > 0:
            ctx.buy_shares = ctx.calc_target_shares(1)
    else:
        # Sell if the next bar is predicted to have a negative return:
        if ctx.preds('macdModel')[-1] < 0:
            ctx.sell_shares =ctx.calc_target_shares(1)
def buy_cmma_cross(ctx):
    if ctx.long_pos() and  ctx.indicator('macd_dif')[-1]-ctx.indicator('macd_dea')[-1] < 0 and ctx.indicator('macd_hist')[-1]<0:
        ctx.sell_shares = ctx.calc_target_shares(1)
        return
    if not ctx.long_pos() and ctx.indicator('macd_dif')[-1]-ctx.indicator('macd_dea')[-1] >= 0  and ctx.indicator('macd_hist')[-1]>0:
        ctx.buy_shares = ctx.calc_target_shares(1)
   
# pb.enable_caches('walkforward_strategy')
# pb.enable_data_source_cache('akshare_cache')

cmma_20 = pb.indicator('cmma_20', cmma,lookback=30)
macd_dif = pb.indicator('macd_dif', lambda data: talib.MACD(data.close)[0])
macd_dea = pb.indicator('macd_dea', lambda data: talib.MACD(data.close)[1])
macd_hist = pb.indicator('macd_hist', lambda data: talib.MACD(data.close)[2])

model_macd = pb.model('macdModel', train_macd, indicators=[macd_dif,macd_dea,macd_hist])
# strategy.add_execution(None, symbols=['600036'],models=model_macd)
# strategy.backtest(train_size=0.5)
# strategy.clear_executions()
strategy.add_execution(hold_long,  symbols=['600036'], models=model_macd)

result = strategy.walkforward(
    windows=10,
    train_size=0.5,
    lookahead=1,
    calc_bootstrap=True
)
# strategy.add_execution(fn=buy_cmma_cross, symbols=['600036'], indicators=[cmma_20,macd_dif,macd_dea,macd_hist])

# result = strategy.backtest(calc_bootstrap=True)
result.positions


Backtesting: 2024-01-01 00:00:00 to 2025-02-03 17:20:23.961046

Loading bar data...
Loaded bar data: 0:00:00 

Computing indicators...


  0% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--
 33% (1 of 3) |########                  | Elapsed Time: 0:00:02 ETA:   0:00:05
100% (3 of 3) |##########################| Elapsed Time: 0:00:02 Time:  0:00:02



Train split: 2024-01-26 00:00:00 to 2024-03-05 00:00:00
600036 R^2=-279.62253370054736
Finished training models: 0:00:00 

Test split: 2024-03-06 00:00:00 to 2024-04-08 00:00:00


  0% (0 of 22) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (22 of 22) |########################| Elapsed Time: 0:00:00 Time:  0:00:00



Train split: 2024-03-06 00:00:00 to 2024-04-08 00:00:00
600036 R^2=-0.8867008242729948
Finished training models: 0:00:00 

Test split: 2024-04-09 00:00:00 to 2024-05-13 00:00:00


...

  0% (0 of 22) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (22 of 22) |########################| Elapsed Time: 0:00:00 Time:  0:00:00



Train split: 2024-11-26 00:00:00 to 2024-12-25 00:00:00
600036 R^2=0.06557222936026097
Finished training models: 0:00:00 

Test split: 2024-12-26 00:00:00 to 2025-01-27 00:00:00


  0% (0 of 22) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
100% (22 of 22) |########################| Elapsed Time: 0:00:00 Time:  0:00:00



Calculating bootstrap metrics: sample_size=100, samples=10000...
Calculated bootstrap metrics: 0:00:00 

Finished backtest: 0:00:03


long_shares  short_shares  close     equity  market_value  \
symbol date                                                                    
600036 2024-03-07         3128             0  32.00  100096.00     100096.00   
       2024-03-08         3128             0  31.67   99063.76      99063.76   
       2024-03-11         3128             0  31.48   98469.44      98469.44   
       2024-03-12         3128             0  31.95   99939.60      99939.60   
       2024-03-13         3128             0  31.52   98594.56      98594.56   
...                        ...           ...    ...        ...           ...   
       2025-01-21         2695             0  40.14  108177.30     108177.30   
       2025-01-22         2695             0  39.45  106317.75     106317.75   
       2025-01-23         2695             0  39.83  107341.85     107341.85   
       2025-01-24         2695             0  40.26  108500.70     108500.70   
       2025-01-27         2695             0  40.65  109551.75     109551.75   

                   margin  unrealized_pnl  
symbol date                                
600036 2024-03-07     0.0          125.12  
       2024-03-08     0.0         -907.12  
       2024-03-11     0.0        -1501.44  
       2024-03-12     0.0          -31.28  
       2024-03-13     0.0        -1376.32  
...                   ...             ...  
       2025-01-21     0.0         2506.35  
       2025-01-22     0.0          646.80  
       2025-01-23     0.0         1670.90  
       2025-01-24     0.0         2829.75  
       2025-01-27     0.0         3880.80  

[217 rows x 7 columns]

In [111]:
result.trades


,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop,mae,mfe
id,,,,,,,,,,,,,,,
1,long,600036,2024-03-07,2024-04-10,31.96,32.44,3128,1501.44,1.50,1501.44,22,68.25,None,-1.30,0.96
2,long,600036,2024-04-12,2024-04-16,32.26,32.71,3142,1413.90,1.39,2915.34,2,706.95,None,-0.20,0.74
3,short,600036,2024-04-16,2024-05-15,32.71,35.06,4,-9.40,-6.70,2905.94,18,-0.52,None,-3.42,0.26
4,long,600036,2024-05-15,2024-06-17,35.06,33.60,2908,-4245.68,-4.16,-1339.74,22,-192.99,None,-2.34,1.94
5,short,600036,2024-06-17,2024-07-17,33.60,33.92,28,-8.96,-0.94,-1348.70,22,-0.41,None,-1.65,0.81
6,long,600036,2024-07-17,2024-07-18,33.92,34.00,2891,231.28,0.24,-1117.42,1,231.28,None,-0.27,0.27
7,short,600036,2024-07-18,2024-07-23,34.00,33.47,17,9.01,1.58,-1108.41,3,3.00,None,-0.30,0.91
8,long,600036,2024-07-23,2024-08-16,33.47,32.98,2948,-1444.52,-1.46,-2552.93,18,-80.25,None,-2.09,0.24
9,short,600036,2024-08-16,2024-09-05,32.98,31.17,6,10.86,5.81,-2542.07,14,0.78,None,-1.00,1.81


In [112]:
result.orders

,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,600036,2024-03-07,3128,NaN,31.96,0.0
2,sell,600036,2024-04-10,3128,NaN,32.44,0.0
3,buy,600036,2024-04-12,3142,NaN,32.26,0.0
4,sell,600036,2024-04-16,3146,NaN,32.71,0.0
5,buy,600036,2024-05-15,2912,NaN,35.06,0.0
6,sell,600036,2024-06-17,2936,NaN,33.60,0.0
7,buy,600036,2024-07-17,2919,NaN,33.92,0.0
8,sell,600036,2024-07-18,2908,NaN,34.00,0.0
9,buy,600036,2024-07-23,2965,NaN,33.47,0.0


In [113]:
result.metrics_df

,name,value
0,trade_count,13.000000
1,initial_market_value,100000.000000
2,end_market_value,112360.840000
3,total_pnl,8480.040000
4,unrealized_pnl,3880.800000
5,total_return_pct,8.480040
6,total_profit,21121.280000
7,total_loss,-12641.240000
8,total_fees,0.000000
9,max_drawdown,-14791.390000


In [3]:
import akshare as ak
df = ak.stock_comment_em()
df = df[df["最新价"] < df["主力成本"]]
# 2. 标准化指标
# 综合得分：越高越好，直接标准化
df["综合得分_标准化"] = (df["综合得分"] - df["综合得分"].min()) / (df["综合得分"].max() - df["综合得分"].min())

# 目前排名：越低越好，反向标准化
df["目前排名_标准化"] = 1 / df["目前排名"]

# 3. 计算综合评分（权重均为 1/3）
df["综合评分"] = (df["综合得分_标准化"] + df["目前排名_标准化"]) / 2

# 4. 按综合评分降序排序，取前10只股票
top_10 = df.sort_values(by="综合评分", ascending=False).head(10)
display(top_10)

  0%|          | 0/11 [00:00<?, ?it/s]

,序号,代码,名称,最新价,涨跌幅,换手率,市盈率,主力成本,机构参与度,综合得分,上升,目前排名,关注指数,交易日,综合得分_标准化,目前排名_标准化,综合评分
1497,1498,300017,网宿科技,13.34,2.22,23.21,57.00,13.474448,0.480141,79.673013,-11,5,94.4,2025-02-07,1.000000,0.200000,0.600000
2058,2059,300613,富瀚微,68.19,3.85,12.57,63.85,68.400667,0.415090,78.030204,-76,15,89.2,2025-02-07,0.955615,0.066667,0.511141
2073,2074,300628,亿联网络,43.10,2.18,2.07,19.79,43.302259,0.302653,77.892033,-70,17,83.2,2025-02-07,0.951882,0.058824,0.505353
4844,4845,688360,德马科技,23.28,1.84,4.67,28.17,23.481054,0.379642,76.969546,4,26,70.4,2025-02-07,0.926958,0.038462,0.482710
4295,4296,603786,科博达,71.65,5.14,1.18,35.78,71.846114,0.238090,76.515550,-49,30,79.6,2025-02-07,0.914692,0.033333,0.474013
2356,2357,300925,法本信息,26.72,1.83,20.89,78.16,26.969655,0.379749,76.408655,-29,32,88.4,2025-02-07,0.911804,0.031250,0.471527
4618,4619,688099,晶晨股份,88.81,1.38,4.46,46.97,90.268386,0.419556,76.205816,6,38,82.0,2025-02-07,0.906324,0.026316,0.466320
4151,4152,603501,韦尔股份,117.10,3.13,4.19,44.97,117.777043,0.485475,75.695839,29,44,93.2,2025-02-07,0.892546,0.022727,0.457636
1301,1302,002850,科达利,120.00,-0.25,5.93,24.03,120.365456,0.370124,75.688674,-51,45,84.8,2025-02-07,0.892352,0.022222,0.457287
4063,4064,603305,旭升集团,17.85,3.54,7.92,38.68,17.941634,0.376593,75.186276,1,52,90.4,2025-02-07,0.878778,0.019231,0.449004


In [114]:
result.bootstrap.conf_intervals

lower     upper
name          conf                     
Profit Factor 97.5%  0.324225  1.309014
              95%    0.343398  1.164945
              90%    0.402658  1.022827
Sharpe Ratio  97.5% -0.349563  0.078558
              95%   -0.333471  0.044335
              90%   -0.284067  0.005258